### Word2Vec 개선
- 말뭉치에 포함된 어휘 수가 많아지면 계산량이 커져 모델의 게산 시간이 너무 오래 걸린다는 단점이 존재함
- Word2Vec 모델에 두 가지 개선사항을 반영하여 모델의 속도 개선을 이루고자 함
 1. 'Embedding'이라는 새로운 게층 도입
 2. '네거티브 샘플링'이라는 새로운 손실 함수 도입
- 예를 들어, 어휘가 100만 개, 은닉층의 뉴런이 100개인 2계층 CBoW 모델일 경우
 * 입력층의 원핫 표현과 가중치 행렬 W_in의 곱 계산
  > Embedding 게층 도입으로 문제 해결 가능
 * 은닉층과 가중치 행렬 W_out의 곱 및 softmax 계층의 계산
  > 네거티브 샘플링 손실 함수 도입으로 문제 해결 가능

#### Embedding 계층
- 가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'을 추출하는 계층으로, Embedding 계층에 단어 임베딩(분산 표현)을 저장함

In [1]:
### 행렬에서 특정 행 추출하기
import numpy as np
W = np.arange(21).reshape(7, 3)
print(W)
print(W[2])
print(W[5])

idx = np.array([1, 0, 3, 0])
print(W[idx])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[15 16 17]
[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [2]:
### Embedding 계층 구현
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    ### Embedding 계층의 순전파는 가중치 W의 특정 행만 추출함
    ### 단순히 가중치의 특정 행 뉴런만을 (아무것도 손대지 않고) 다음 층으로 흘려보낸 것
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    ### 역전파에서는 앞 층(출력 측 층)으로부터 전해진 기울기를 다음 층(입력 측 층)으로 그대로 흘려주면 됨
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
#         dW[self.idx] = dout   ### idx의 원소가 중복될 때 문제 발생
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
#         np.add.at(dW, self.idx, dout)   ### 사용가능한 다른 방법 (효율이 더 좋음)
        return None

#### 네거티브 샘플링
- Softmax 대신 네거티브 샘플링을 이용하면 어휘가 아무리 많아져도 계산량을 낮은 수준으로 일정하게 억제할 수 있음

- 은닉층 이후 계산의 문제점
 * 예를 들어, 어휘가 100만 개, 은닉층의 뉴런이 100개인 CBoW 모델에 Embedding 계층을 도입한 경우
  > - 은닉층의 뉴런과 가중치 행렬 W_out의 곱 : 거대한 행렬의 곱을 계산하는 문제 <br>
  > - Softmax 계층의 계산 : 어휘가 많아지면 softmax의 계산량도 증가함

- 다중 분류에서 이진 분류로 
 * 다중 분류 : 맥락이 주어졌을 때 정답이 되는 단어 하나를 선택하는 문제
  > ex. 맥락이 'you'와 'goodbye'일 때, 타깃 단어는 무엇입니까?
 * 이진 분류 : 'Yes/No'로 답하는 문제
  > ex. 맥락이 'you'와 'goodbye'일 때, 타깃 단어는 'say'입니까?
 * 은닉층과 출력 측의 가중치 행렬의 내적은 'say'에 해당하는 열(단어 벡터)만을 추출하고, 그 추출된 벡터와 은닉층 뉴런과의 내적을 계산하면 됨

In [3]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)   ### 내적 계산
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

- 시그모이드 함수와 교차 엔트로피 오차
 * 흔히 이진 분류 신경망에서는 점수에 시그모이드 함수를 적용해 확률로 변환하고, 손실을 구할 때는 교차 엔트로피 오차를 사용함
  > 다중 분류 : 소프트맥스 함수 & 교차 엔트로피 오차 <br>
  > 이진 분류 : 시그모이드 함수 & 교차 엔트로피 오차

- 네거티브 샘플링 기법
 * 적은 수의 부정적 예를 샘플링해서 사용하는 것
  > - 다중 분류 문제를 이진 분류로 다루려면 '정답(긍정적 예)'과 '오답(부정적 예)' 각각에 대해 바르게 (이진)분류할 수 있어야 함
  > - 하지만 모든 부정적 예를 대상으로 하는 방법은 어휘 수가 늘어나면 감당할 수 없어 근사적인 해법으로 네거티브 샘플링을 사용함
 * 네거티브 샘플링은 각각의 데이터(긍정적 예와 샘플링된 부정적 예)의 손실을 더한 값을 최종 손실로 함

- 네거티브 샘플링과 샘플링 기법
 * 부정적 예를 샘플링하는 방법
  > - 말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어(희소 단어)를 적게 추출함
  > - 각 단어의 출현 횟수에 대한 확률분포를 토대로 단어를 샘플링함

In [4]:
### 0에서 9까지의 숫자 중 하나를 무작위로 샘플링
print(np.random.choice(10))
print(np.random.choice(10))

### words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
print(np.random.choice(words))

### 5개만 무작위로 샘플링(중복 있음)
print(np.random.choice(words, size=5))

### 5개만 무작위로 샘플링(중복 없음)
print(np.random.choice(words, size=5, replace=False))

### 획률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

9
8
you
['I' '.' 'hello' '.' 'goodbye']
['.' 'hello' 'goodbye' 'you' 'I']
you


In [5]:
### 네커티브 샘플링에서는 기본 확률분포에 0.75 제곱하는 것을 권고함
### 이는 출현 확룰이 낮은 단어를 '버리지 않기'위함
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [6]:
import collections
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]
        negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)
        for i in range(batch_size):
            p = self.word_p.copy()
            target_idx = target[i]
            p[target_idx] = 0
            p /= p.sum()
            negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        return negative_sample
    
    
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[4 3]
 [1 2]
 [2 1]]


In [7]:
### 네거티브 샘플링 구현
from common.layers import Embedding, SigmoidWithLoss

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size+1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size+1)]
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        ### 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        ### 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1+i].forward(h, negative_target)
            loss += self.loss_layers[1+i].forward(score, negative_label)
        
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        return dh

In [8]:
### CBoW 모델 구현
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        ### 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        ### 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        ### 모든 가중치와 기울기를 배열에 모음
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        ### 인스턴스 변수에 단어의 분산 표현을 저장
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [9]:
### CBoW 모델 학습
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target, to_cpu
from dataset import ptb

### 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 1

### 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)

### 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

### 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

### 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = './output/cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 3[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 5[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 6[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 8[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 10[s] | 손실 3.79
| 에폭 1 |  반복 141 / 9295 | 시간 12[s] | 손실 3.64
| 에폭 1 |  반복 161 / 9295 | 시간 13[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 15[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 17[s] | 손실 3.27
| 에폭 1 |  반복 221 / 9295 | 시간 18[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 20[s] | 손실 3.10
| 에폭 1 |  반복 261 / 9295 | 시간 22[s] | 손실 3.02
| 에폭 1 |  반복 281 / 9295 | 시간 24[s] | 손실 2.95
| 에폭 1 |  반복 301 / 9295 | 시간 25[s] | 손실 2.92
| 에폭 1 |  반복 321 / 9295 | 시간 27[s] | 손실 2.86
| 에폭 1 |  반복 341 / 9295 | 시간 29[s] | 손실 2.82
| 에폭 1 |  반복 361 / 9295 | 시간 30[s] | 손실 2.81
| 에폭 1 |  반복 381 / 9295 | 시간 32[s] | 손실 2.79
| 에폭 1 |  반복 401 / 9295 | 시간 34[s] | 손실 2.74
| 에폭 1 |  반복 421 / 9295 | 시간 36[s] | 손실 2.74
| 에폭 1 |  반복 441 / 929

<Figure size 640x480 with 1 Axes>

In [10]:
from common.util import most_similar

pkl_file = './output/cbow_params.pkl'
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    
word_vecs = params['word_vecs']
word_to_id = params['word_to_id']
id_to_word = params['id_to_word']

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.98193359375
 i: 0.9775390625
 why: 0.97705078125
 'm: 0.95703125
 're: 0.955078125

[query] year
 month: 0.91943359375
 week: 0.88330078125
 earlier: 0.8798828125
 robust: 0.83984375
 quarter: 0.83447265625

[query] car
 book: 0.9697265625
 dialing: 0.966796875
 strike: 0.966796875
 corporation: 0.96142578125
 virus: 0.9609375

[query] toyota
 royal: 0.9453125
 steel: 0.94287109375
 fuel: 0.939453125
 mobile: 0.93798828125
 wcrs: 0.93798828125


In [11]:
from common.util import analogy
print(analogy('king', 'man', 'queen', word_to_id, id_to_word, word_vecs))
print(analogy('take', 'took', 'go', word_to_id, id_to_word, word_vecs))
print(analogy('car', 'cars', 'child', word_to_id, id_to_word, word_vecs))
print(analogy('good', 'better', 'bad', word_to_id, id_to_word, word_vecs))


[analogy] king:man = queen:?
 share: 5.1328125
 spokesman: 4.69140625
 know: 4.390625
 problem: 4.33984375
 lot: 4.31640625
None

[analogy] take:took = go:?
 share: 4.87109375
 cents: 4.46484375
 yield: 4.39453125
 yen: 3.904296875
 composite: 3.568359375
None

[analogy] car:cars = child:?
 yield: 4.06640625
 cents: 3.94140625
 million: 3.931640625
 revenue: 3.751953125
 president: 3.66015625
None

[analogy] good:better = bad:?
 do: 4.46875
 n't: 4.390625
 know: 4.1953125
 composite: 4.1015625
 you: 4.03125
None


- 유추 문제에 의한 단어 벡터의 평가 결과
 * 모델에 따라 정확도가 다름 -> 말뭉치에 따라 적합한 모델 선택
 * 일반적으로 말뭉치가 클수록 결과가 좋음 -> 항상 데이터가 많은 게 좋음
 * 단어 벡터 차원 수는 적당한 크기가 좋음 -> 너무 커도 정확도가 나빠짐

#### Conclusion
- Embedding 계층은 단어의 분산 표현을 담고 있으며, 순전파 시 지정한 단어 ID의 벡터를 추출함
- Word2Vec은 어휘 수의 증가에 비례하여 계산량도 증가하므로, 근사치로 계산하는 빠른 기법을 사용하면 좋음
- 네거티브 샘플링은 부정적 예를 몇 개 샘플링하는 기법으로, 이를 이용하면 다중 분류를 이진 분류처럼 취급할 수 있음
- Word2Vec으로 얻은 단어의 분산 표현에는 단어의 의미가 녹아들어 있으며, 비슷한 맥락에서 사용되는 단어는 단어 벡터 공간에서 가까이 위치함
- Word2Vec의 단어의 분산 표현을 이용하면 유추 문제를 벡터의 뎃셈과 뺄셈으로 풀 수 있음
- Word2Vec은 전이 학습 측면에서 특히 중요하며, 그 단어의 분산 표현은 다양한 자연어 처리 작업에 이용할 수 있음